# SIXT33N Project  Phase 5: Advanced Controls (and more!)

### EECS 16B: Designing Information Devices and Systems II, Fall 2021

Written by Vivek Athalye and Andrew Blatner (2017). Revised by Peter Schafhalter (2019), Hossein Najafi (2021), Yi-Hsuan Shih (2021).

## Table of Contents

* [Introduction](#intro)
* [Part 1: Analyzing a Turn](#part1)
* [Part 2: Implementing Turns](#part2)
* [Part 3: Recording Your Test Word](#part3)
* [Part 4: Arduino Implementation of PCA Classify](#part4)
* [Part 5: Checkoff](#part5)

## <span style="color:#ba190f"> You need to run the Python scripts one by one, or errors will show up as they rely on the variables defined sequentially!!

## <span style="color:#ba190f"> DO NOT include units when submitting your answers on Gradescope! ONLY include the numerical value rounded to the number of decimal places specified in each question, in the units specified in the question. DO NOT include letters, words, etc. If a question involves entering an answer that is not a numerical value, the format of the answer will be clearly specified in that question.

<a id='intro'></a>
## Introduction

This week, we will first revisit controls. Recall that our control scheme for SIXT33N modeled each wheel in the following way:

$$d[n+1] = d[n] + \theta u[n] - \beta$$

The open-loop model has an input $u[n]$, corresponding to the PWM level, and outputs the distance for that wheel. We implemented a closed-loop controller by using the desired velocity and delta, or the difference between the wheel distances, $$\delta[n] = d_L[n] - d_R[n]$$.

In this phase, we will implement turns by modifying the closed-loop control scheme we developed to make SIXT33N drive straight.

Then, we will implement and port our PCA classifier from the iPython notebook to a file that can be run on the Arduino. This will allow us to integrate PCA with the rest of the project.


<a id='part1'></a>
## <span style="color:navy">Part 1: Analyzing a Turn</span>

### Materials
- SIXT33N Module in TinkerCAD
  - Model Link to be used in the entire lab: [TinkerCAD Model](https://www.tinkercad.com/things/cQ4lrTpatp3) 
- Closed-loop control scheme

### We want to turn with a specific radius.  Which open-loop control input would lead to turning?  
During open-loop control, we sought to drive the car straight at our desired velocity $v^*$.  

We used our model of the car ($\theta$, $\beta$) to calculate the open-loop control input which drives the car straight in the absence of perturbations:  

$$u^{OL}_L = \frac{v^*+\beta_L}{\theta_L} $$
$$u^{OL}_R = \frac{v^*+\beta_R}{\theta_R} $$

Is it possible to calculate an open-loop control input which would result in turning with a desired radius?


### We have been turning during Closed-Loop Control!
We can get a clue about turning by considering how we did closed-loop control. We perform feedback control of SIXT33N in order to minimize the difference between the two wheels ($\delta[n] = d_L[n] - d_R[n]$). When perturbations cause one wheel to get ahead of the other resulting in non-zero $\delta$, feedback control turns the car to correct the error.  

Let's walk through this.  

Our feedback control policy is: 
$$u_L[n] = u_L^{OL} - \frac{f_L}{\theta_L}\delta[n]$$

$$u_R[n] = u_R^{OL} + \frac{f_R}{\theta_R}\delta[n]$$


Let's say the right wheel has moved further than the left ($d_R[n] > d_L[n]$), resulting in a negative $\delta[n]$. A negative $\delta[n]$ results in a reduction of $u_R[n]$, and an increase of $u_L[n]$. But what does this mean physically?

If the right wheel is getting ahead of the left one, the left wheel will need to move a little faster and the right wheel a little slower for the wheels to even back out. Thus, feedback control corrects errors by turning to compensate for them.  

So $\delta \ne 0$ can be used to turn the car. Let's get more specific.  


### Turning via reference tracking

We would like the car to turn with a specified radius $r$ and speed $v^*$. The controller's unit for distance is encoder ticks, but each tick is approximately 1 cm of the wheel's circumference.

To turn, we want $\delta$ to change at a particular rate. Without loss of generality, we'll analyze a right turn, corresponding to increasing $\delta$. For a left turn, we simply negate $\delta$. Our goal is to generate a reference from the desired $r$ and $v^*$ for the controller to follow. This reference will be a function of the controller's time-step.

Use the following variables:
- $n$ - time-step
- $r$ [cm] - turn radius of the center of the car; 1 cm ~ 1 encoder tick
- $d$ [ticks] - distance traveled by the center of the car
- $l$ [cm] - distance between the centers of the wheels; 1 cm ~ 1 encoder tick
- $\omega$ [rad/tick] - angular velocity
- $\theta$ [rad] - angle turned

Inspect the following diagram:

<center>
<img width="400px" src="images/turning_figure.png">
</center>



From this geometry, can you write $\delta[n]$ in the following form?
$$\delta[n] = f(r,v^*,l,n)$$
Hint 1: $$\text{Arc length} = \text{radius} \times \text{central angle}$$
<br>
Hint 2: $$\text{Distance travelled} = \text{velocity} \times \text{time}$$



<span style="color:#075a04"> **1. What is the relationship between $\delta[n]$ and $v^*$ if $l$ and the turning radius $r$ are kept constant? <span style="color:#ba190f">**

< YOUR ANSWER ON GRADESCOPE >    

<span style="color:#075a04"> **2. How does $\delta[n]$ change as a function of the number of time-steps ($n$) if the car is driving at a constant $v^*$ and $l$ and the turning radius $r$ are kept constant? <span style="color:#ba190f">**

< YOUR ANSWER ON GRADESCOPE >    

<span style="color:#075a04"> **3. If the car's speed doubles and its turning radius is also increased by 50%, how does the rate of $\delta[n]$ compare to before? Find the ratio of $\LARGE \frac{\delta_{new}[n]}{\delta_{old}[n]}$.<span style="color:#ba190f">**

< YOUR ANSWER ON GRADESCOPE >    

<a id='part2'></a>
## <span style="color:navy">Part 2: Implementing Turns</span>

### Materials
- TinkerCAD Module
- Closed-loop control scheme
- Plan from Task 1

### Sampling periods 
 
In our implementation, the control loop and the data collection have different periods, $T_c$ and $T_d$ respectively, and therefore different sampling frequencies, $F_c$ and $F_d$ respectively.  $F_c$ is an integer multiple of $F_d$: $m=\frac{F_c}{F_d} \in \mathbb{Z}^{+}$

To ensure the car is travelling at the same ground velocity in both cases, we want to have a constant distance, measured in ticks, travelled per second, or $\frac{ticks}{s}$.

$F_c$, with units [$\frac{sample_c}{s}$], is the number of times the closed-loop is sampled every second, and $F_d$, with units [$\frac{sample_d}{s}$], is the number of times data collection was sampled per second. Let $v^*_c$ be the velocity of closed-loop, and $v^*$ be the velocity of the data collection. $v^*_c$ has the units of [$\frac{ticks}{sample_c}$] and $v^*$ has units [$\frac{ticks}{sample_d}$].

To get a constant $\frac{ticks}{s}$, we need $v^*_c * F_c = v^* * F_d$.

Solving for $v^*_c$, we get $v^*_c = v^* * \frac{F_d}{F_c} = \frac{v^*}{m}$

We use $T_c=100$ms, $T_d=500ms$, so $F_c=10$, $F_d=2$, so $m=5$.

**<span style='color:red'> In your delta_reference function, use $\frac{v^*}{m}$ instead of $v^*$. Do not replace the original $v^*$ parameter defined in `CODE BLOCK CON1`!</span>**

### Implementation
- Open `turning.ino`.
- We have already filled out **`CODE BLOCK CON1`** for you to make sure that there is no error propagation from the previous lab. Please fill out **`CODE BLOCK CON2`** and **`CODE BLOCK CON3`**.
    - Make sure to update pin numbers if you've made changes.
    - Setting `delta_ss` helps your car's driving converge to straight sooner! For `delta_ss` corresponding to the f values you choose, you can check the steady state error correction section in **Controls** lab. 
- **Complete the function `delta_reference` in the preprogrammed path under `CODE BLOCK CON4`**.
    - Look at `CODE BLOCK CON0` to see how this is used!
    - Use the variables `TURN_RADIUS`, `v_star`, `CAR WIDTH`, and `n`.
    - `delta_ss` is used in `CODE BLOCK CON0`, so don't include it here.

When you're done, try it out! SIXT33N should go straight, turn left, go straight, and turn right. To show how the car moves in TinkerCAD, set **<span style='color:red'> mode = 0 </span>** in line 43 and observe the two waveforms being plotted in the serial plotter. The blue curve corresponds to the x-position of the car, and the orange line corresponds to the y-position of the car. The trajectory being represented is the shape below. The x and y positions of this trajectory are also shown over time. You should be able to observe a similar trajectory in the serial plotter in TinkerCAD.

In the Serial plotter, the <span style="color:blue">BLUE</span> line corresponds to the <span style="color:blue">X</span> position of the car and the <span style="color:#EACE09">YELLOW</span> line to the <span style="color:#EACE09">Y</span> position of the car.

<img width=700px src="images/car.png">
<img width=700px src="images/car2.png">

### Visualization of the Trajectory 
To visualize the trajectory, we are going to copy the simulation data from the serial monitor in TinkerCad, save it as a .txt file, and then process it in the following code block. Please follow the steps below.

- Set `mode=1` on line 43 in `turning.ino`. In this mode, the serial monitor would only print the value every `SAMPLE_CONST` of the sample. It is set to 10 to allow all the sampled data from 4 commands be printed in the serial monitor at the end of the simulation. A large number `10000, 10000` is inserted at the end of the commands so that we can recognize the end of the command in the python code. 
- After running the simulation, copy the data from the serial monitor and save to `trajactory.txt` file in the folder where you run the Jupyter notebook. Then, run the following code block to observe the trajectory of the car.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_car_traj(file_name, command):
    color = {'DRIVE_STRAIGHT': 'blue', 'DRIVE_LEFT': 'orange', 'DRIVE_RIGHT': 'green'}
    data = np.genfromtxt(file_name, dtype=np.float, delimiter=",")
    
    fig = plt.figure(figsize=(10, 7))

    x = data[:, 0]
    y = data[:, 1]
    
    dx, _ = np.where(data==[10000, 10000])
    dx = [dx[2 * idx] for idx in range(dx.shape[0] // 2)]
    
    ax = fig.add_subplot(111)

    N = x.shape[0]
    ax.set_aspect('equal', adjustable='box')
    plt.xlabel("x")
    plt.ylabel("y")
    
    for idx in range(len(dx)):
        start_idx = 0 if idx == 0 else dx[idx - 1] + 1
        end_idx = -3 if idx == len(dx)-1 else dx[idx] - 1
        end_point = -2 if idx == len(dx)-1 else dx[idx] + 1
        ax.quiver(x[start_idx:end_idx], y[start_idx:end_idx],  np.append(x[start_idx + 1:end_idx], x[end_point])-x[start_idx:end_idx],  np.append(y[start_idx + 1:end_idx], y[end_point])-y[start_idx:end_idx], scale_units='xy', angles='xy', scale=1, color=color[command[idx]])
    plt.grid()
    plt.legend(command, loc="best")
    plt.title('Trajectory of the car')
    plt.show()
    print(f'final position (x, y): ({x[-2]}, {y[-2]})')
    return 

plot_car_traj('trajectory.txt', ('DRIVE_STRAIGHT', 'DRIVE_LEFT', 'DRIVE_STRAIGHT', 'DRIVE_RIGHT'))

<span style="color:#075a04"> **4. If the car's speed $v^*$ is set to be $202.343$, and both $f_{left}$ and $f_{right}$ are set to 0.2, what is the final value of the X position of the car? The coordinates being printed in the serial plotter are formatted as (X position, Y position).<span style="color:#ba190f">**

< YOUR ANSWER ON GRADESCOPE >    

<span style="color:#075a04"> **5. In the same case as Q4, what is the final value of the Y position of the car?<span style="color:#ba190f">**

< YOUR ANSWER ON GRADESCOPE >    

<span style="color:#075a04"> **6. Change the length of the 3rd command from 8000 to 4000. You can do so by changing the line `int run_times[NUM_COMMANDS] = {8000, 12600, 8000, 12600};` to `int run_times[NUM_COMMANDS] = {8000, 12600, 4000, 12600};`. If the car's speed $v^*$ is set to be $202.343$, and both $f_{left}$ and $f_{right}$ are set to 0.2, what is the final value of the X position of the car?<span style="color:#ba190f">**

< YOUR ANSWER ON GRADESCOPE >    

<span style="color:#075a04"> **7. In the same case as Q6, what is the final value of the Y position of the car? <span style="color:#ba190f">**

< YOUR ANSWER ON GRADESCOPE >    

**8. Change the length of the 2nd and 4th commands from 12600 to 25200. You can do so by changing the line `int run_times[NUM_COMMANDS] = {8000, 12600, 8000, 12600};` to `int run_times[NUM_COMMANDS] = {8000, 25200, 8000, 25200};`. If the car's speed $v^*$ is set to be $202.343$, and both $f_{left}$ and $f_{right}$ are set to 0.2:**
    
<span style="color:#075a04"> **8a. What do you observe in the trajectory of the car? Which of the following applies to the trajectory the car is taking? *Hint: Run simulation with mode=1, copy the data and paste it to `
trajectory_q8.txt` in this folder. You should be able to see the trajectory in the following code block.*<span style="color:#ba190f">** 

  < YOUR ANSWER ON GRADESCOPE >    

In [ ]:
plot_car_traj('trajectory_q8.txt', ('DRIVE_STRAIGHT', 'DRIVE_LEFT', 'DRIVE_STRAIGHT', 'DRIVE_RIGHT'))

<span style="color:#075a04"> **8b. What is the final value of the X position of the car?<span style="color:#ba190f">**

< YOUR ANSWER ON GRADESCOPE >    

<span style="color:#075a04"> **8c. What is the final value of the Y position of the car?<span style="color:#ba190f">**

< YOUR ANSWER ON GRADESCOPE >    

<a id='part3'></a>
## <span style="color:navy">Part 3: Recording Your Test Word</span>


### Materials
- Microphone



This section will walk you through implementing your classification algorithm on the Arduino. In addition, you will need to transfer the PCA vectors and mean to the Microcontroller. Re-visit your PCA lab Jupyter notebook and use the next few lines to print and format your PCA vectors to have them ready to be copied into the TinkerCAD environment to initialize the PCA vector variables on the microcontroller. You should copy and paste the following lines in the approrpriate place in your PCA Jupyter notebook and initialize the variable `pca_num` properly to print the first and then the second PCA vector. The following code for instance prints the first PCA vector, assuming `Vt` is  the `Vt` of the SVD decomposition during PCA:

In [ ]:
a = '={'
v = Vt
pca_num = 0

for i in v[pca_num,:].T:    # change pca_num to print the corresponding vector
    a += str(float(int(i*10000))/10000) + ', '
a = a[:-2]+"};"
print("PCA vector")
print(a)

**<span style='color:red'> Optional if you already have test sets from the SVD/PCA lab!</span>**
<a id='part2'></a>

**Use the next section to record your own test word (only one word, not 4 words like before!) and process it using the following code. The last part of the code formats the audio vector ready to be copied and pasted in TinkerCAD to initialize the relevant variable (`result`). You will need it for Part 4.**


You are going to **record your own `wav` files with one of the four words you chose in PCA.**

"Good" audio data has a high signal to noise ratio. Recording words while far away from the microphone may cause your intended word to blend in with background noise. However "oversaturation" of the audio signal (speaking too loudly and/or too closely into the mic) will distort the signal. 

To record your own `wav` files, we suggest you use the [online recorder](https://online-voice-recorder.com). 

**For the chosen word, do the following:**
1. Click on record button on the webpage, and come back to this page. 
2. **Run the following script, when you see the number 1 appear, say the word you want to record.**
    - The recording window is 2 second. You want to finish the word in this period.
    - **Pronounce the word consistently.** This will help you collect data that is less "noisy" which will result in better classification.
    
    
3. Once you've recorded the word, the script will stop automatically. **Then go back to the webpage and stop the recording.** You should get a webpage like the following.

<center>
<img width="800px" src="images/recording.png">
</center>

4. Change the start time to be near your sample and near the end time. **You want to remove the noise before and after the sample.** The total recording time should be around 2 seconds.

5. When you save the file, it is the `mp3` type. Use the [audio converter](https://online-audio-converter.com) to convert it to `wav` file with the options shown as below.

<center>
<img width="600px" src="images/convert_to_wav.png">
</center>

6. Tweak `length`, `pre_length` and `threshold` to make sure the samples are aligned and distinguishable in time waveforms.

If you cannot get access to the website, you can record with your phone. But you need to convert it to `wav` file and also clip it to remove the noise singnals before the first sample and after the last sample. Another website to convert the audio files are [Audio Converter](https://www.freeconvert.com/audio-converter) and the website to trim the `wav` file is [mp3cutter](https://www.mp3cutter.com). You can also find other useful websites to do this.

In [ ]:
import utils
record_time = 2   # record time for each sample of the word

utils.recording_timer(counts, record_time)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import csv
import utils
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

counts = 1

def get_snippets(data, length, pre_length, threshold):
    """Attempts to align audio samples in data.
    
    Args:
        data (np.ndarray): Matrix where each row corresponds to a recording's audio samples.
        length (int): The length of each aligned audio snippet.
        pre_length (int): The number of samples to include before the threshold is first crossed.
        threshold (float): Used to find the start of the speech command. The speech command begins where the
            magnitude of the audio sample is greater than (threshold * max(samples)).
    
    Returns:
        Matrix of aligned recordings.
    """
    assert isinstance(data, np.ndarray) and len(data.shape) == 2, "'data' must be a 2D matrix"
    assert isinstance(length, int) and length > 0, "'length' of snippet must be an integer greater than 0"
    assert 0 <= threshold <= 1, "'threshold' must be between 0 and 1"
    snippets = []

    # Iterate over the rows in data
    for recording in data:
        # Find the threshold
        recording_threshold = threshold * np.max(recording)

        # Figure out when interesting snippet starts
        i = pre_length
        while recording[i] < recording_threshold:
            i += 1
            
        snippet_start = min(i - pre_length, len(recording) - length)
        snippet = recording[snippet_start:snippet_start + length]

        # Normalization
        snippet = snippet / np.sum(snippet)
        
        snippets.append(snippet)

    return np.vstack(snippets)


# Load data from wav
word_raw_test = utils.read_wav("test_word/test_word_new.wav", counts)


# Get snippets
length = 40      # Default: 40     # Keep for example wav files, adjust this with your recordings
pre_length = 5   # Default: 5      # Keep for example wav files, adjust this with your recordings
threshold = 0.5  # Default:  0.5   # Keep for example wav files, adjust this with your recordings

word_processed_test = get_snippets(word_raw_test, length, pre_length, threshold)
plt.plot(word_processed_test.T)
plt.show()
plt.figure()

a = '={'
v = word_processed_test[0,:]


for i in v.T:
    a += str(float(int(i*10000))/10000) + ', '
a = a[:-2]+"};"
print(a)

##### <a id='part4'></a>
## <span style="color:navy">Part 4: Arduino Implementation of PCA Classifier</span>


### Materials
- Recorded word
- TinkerCAD Model



Your last step will be to implement your <b>data processing</b> and <b>classification</b> (just the projection, not the PCA) in the Arduino sketch <b>`classify.ino`</b>. Since the Arduino environment does not have as many built-in functions as Python, you might have to write out the functions yourself. For example, a dot product should be written as:

`float result = 0;`<br/>
`for (i=0; i<LENGTH; i++){`<br/>
&emsp; `result += vector1[i]*vector2[i];`<br/>
`}`

For debugging purposes, printing to the serial plotter looks like the line below.

<code>Serial.println("I'm being printed!");</code>

There are 3 code blocks (`PCA1/2/3`) that you need to modify. <b>You should not have to change anything else outside these marked code blocks.</b> 

**`CODE BLOCK PCA1`**
- Copy `SNIPPET_SIZE`, `PRELENGTH` and `THRESHOLD` from the SVD/PCA Jupyter notebook.
- Read the following to set `EUCLIDEAN_THRESHOLD` and `LOUDNESS_THRESHOLD`.

**`EUCLIDEAN_THRESHOLD` filters the classification depending on a sample's distance from the closest centroid.** If the L2 norm (distance) is larger than the threshold, your classification algorithm should simply ignore it and wait for the next sample. Look at the plot of sample data and the centroids from the PCA Notebook and approximate a radius around the centroids that capture most of the data. **Try to be conservative - it's better to not classify than to misclassify.**

**`LOUDNESS_THRESHOLD` filters the classification depending on the amplitude of the recorded data.** If the recorded data is too soft, we do not want to classify it as it is probably noise. Since the loudness unit is arbitrary, start by using `700`. Later, if the Arduino classifies noise, increase this constant. If it misses a lot of speech (i.e. thinks your word is noise), decrease this constant. This variable is used internally in the enveloping function.

**`CODE BLOCK PCA2`**
- Copy the PCA vectors, mean, and centroids from the SVD/PCA Jupyter Notebook.
- If you need more than 2 principal components, add a new `pca_vec3` array. 
- Note: Using more principal components increases the dimensionality of the centroids and projections.

**`CODE BLOCK PCA3`**
- This is the actual classification algorithm.
- Initialize the variable `result` with the processed audio of your word of choice, or use one of the samples already included in the code.
- Project this data onto your new PCA basis.
    - Remember to subtract the mean vector, before doing a dot product for each basis.
    - Use the variables `proj1` and `proj2` to store the projection results.
- Classify the projections using the centroids.
    - Find the distance between the projected data point and each centroid using the function `l2_norm` (for 2 principal components) or `l2_norm3` (for 3 principal components). Look up the function definition in the sketch.
    - Out of the 4 centroids, find the one with the smallest L2 norm.
    - Verify this distance is less than `EUCLIDEAN_THRESHOLD`.
- Print the classification to the serial monitor.

Now upload the sketch, open the serial monitor, and run. Observe how the Arduino classifies your word.

**<span style="color:red">If the Arduino does not classify as well as you think it should, remember to play with the `EUCLIDEAN_THRESHOLD` and `LOUDNESS_THRESHOLD` variables.</span>** To debug the sketch, you can also print out any of the variables you have used. 

Voila! Your SIXT33N can recognize your words!

**<span style="color:red">Summary TODO</span>**
- **<span style="color:red">Fill in `CODE BLOCK PCA1`: Fill out `SNIPPET_SIZE`, `PRELENGTH`, `THRESHOLD`, `EUCLIDEAN_THRESHOLD` and `LOUDNESS_THRESHOLD`</span>**
- **<span style="color:red">Fill in `CODE BLOCK PCA2`: Copy the principal components, mean vector, and centroids from the SVD/PCA notebook</span>**
- **<span style="color:red">Fill in `CODE BLOCK PCA3`: Do the actual classification.</span>**

**Try all 4 words as your test word and make sure you are able to get enough accurary on the Arduino when classifying those words.**

<a id='part5'></a>
## <span style="color:#ba190f">Part 5: CHECKOFF </span> 
-----
    
### For Checkoff:

- Submit your answer to the questions of **Project Sim 5: Advanced Controls** assignment on Gradescope.
- This lab's Gradescope assignment is due on Wednesday 11/24 at 11:59PM PT. Like EECS 16B HW, the lab has a 24 hour grace period with no penalty to accomodate any technical difficulties. The lab can be submitted up to one week late (up until Wednesday 11/31 at 11:59PM PT, with no grace period) for 50% credit. After that, no late submissions will be accepted.

### Make sure your circuits are saved properly in Tinkercad. You will need them in the next project phase!

### Remember what each part of your circuit is for by recording this information in a Google doc or somewhere else safe. You will need to write a summary for your final lab report.